# Predictive Analysis - Current Stint Penalty Minutes to Previous Stint Comparison

Age, percent through career, teammates of same nationality, and teammates of same nationality vs previous stint will be used to predict whether the current stint will have more or fewer penalty minutes per game compared to the previous stint.


## Load Data

In [6]:
import pandas as pd

df = pd.read_csv('Data/Gold/main.csv')

df.head()


,playerID,year,stint,tmID,pos,GP,G,A,Pts,PIM,...,SHA_per_game_score,GWG_per_game_score,SOG_per_game_score,award_count_per_game_score,+/-_score,stint_score,stint_vs_prev_stint,teammates_same_nationality,tsm_vs_prev_stint,age
0,abdelju01,2009,1,DET,L,50.0,3.0,3.0,6.0,35.0,...,-0.707107,-1.409907,-0.248378,0.0,-1.282503,-1.315045,NaN,3,NaN,22.0
1,abdelju01,2010,1,DET,L,74.0,7.0,12.0,19.0,61.0,...,1.414214,0.800463,1.329897,0.0,1.157381,1.108070,1.0,3,0.0,23.0
2,abdelju01,2011,1,DET,L,81.0,8.0,14.0,22.0,62.0,...,-0.707107,0.609444,-1.081518,0.0,0.125122,0.206975,0.0,1,-1.0,24.0
3,abelcl01,1926,1,NYR,D,44.0,8.0,4.0,12.0,78.0,...,0.000000,0.000000,0.000000,0.0,0.000000,2.257283,NaN,0,NaN,26.0
4,abelcl01,1927,1,NYR,D,23.0,0.0,1.0,1.0,28.0,...,0.000000,0.000000,0.000000,0.0,0.000000,-0.554457,0.0,0,0.0,27.0


## Feature Design - Penalty Minutes vs Previous Stint

Exploratory Data Analysis revealed that there was a slight indication of a linear correlation between the number of teammates of the same nationality a player has and the number of penalty minutes per game they will be assessed. We seek to predict whether an increase or decrease in their penalty minutes per game will occur in a given stint, using the aforementioned input features.

In [7]:

X = df[['age', 'percent_through_career', 'teammates_same_nationality', 'tsm_vs_prev_stint']]

print(X.head())

    age  percent_through_career  teammates_same_nationality  tsm_vs_prev_stint
0  22.0                0.019139                           3                NaN
1  23.0                0.258373                           3                0.0
2  24.0                0.612440                           1               -1.0
3  26.0                0.000000                           0                NaN
4  27.0                0.132132                           0                0.0


## Random Forest Analysis

## Model 2 - AdaBoost

## Compare Performance

## Hyperparameter Experimentation

## Summary